In [2]:
from moviepy.editor import *
import gizeh as gz
from gtts import gTTS
import wikipedia
from skimage.io import imread, imsave
from skimage.transform import resize
from skimage.util import img_as_ubyte
import time
from datetime import datetime

In [3]:
def render_text(t):
    surface = gz.Surface(1920, 1080, bg_color=BLACK_GIZEH)
    text = gz.text(
        title, fontfamily="Helvetica",
        fontsize=120, fontweight='bold', fill=WHITE_GIZEH, xy=(960, 400))
    text.draw(surface)
    return surface.get_npimage()

#### Default Parameters:

In [4]:
VIDEO_SIZE = (1920, 1080)
IMG_SHAPE = (540, 960)

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
WHITE_GIZEH = (1, 1, 1)
BLACK_GIZEH = (0, 0, 0)

IMG_DISPLAY_DURATION = 10    #duration, in seconds, to display each image

#### Loading Wikipedia page, extracting transcript, setting folder paths:

In [63]:
page_name = "Badger"
page = wikipedia.page(page_name)
title = page.title
script = page.content[:100] 
### Here would be the place to parse script
print("Video Title: ", title)
# print(script)

Video Title:  Badger


In [64]:
# Folder paths
IMG_FOLDER = 'images/' + title + "/"
RESIZE = IMG_FOLDER + "/resize"
AUDIO_FOLDER = 'audio/'
VID_FOLDER = 'videos/'

AUDIO_PATH = AUDIO_FOLDER + title + ".mp3"
VID_PATH = VID_FOLDER + title + ".mp4"

for folder in [IMG_FOLDER, RESIZE, AUDIO_FOLDER, VID_FOLDER]:
    if not os.path.exists(folder):
        os.makedirs(folder)

#### Converting text-to-speech, creating MoviePy AudioClip:

In [65]:
tts = gTTS(script, lang='en')
tts.save(AUDIO_PATH)
audio_clip = AudioFileClip(AUDIO_PATH)
DURATION = audio_clip.duration
print("Audio Duration: ", time.strftime('%H:%M:%S', time.gmtime(DURATION)))

Audio Duration:  00:00:06


#### Resizing images.

Original images remain unchanged, resized copies are saved to './imgs/[title]/resize/'

In [68]:
final_img_paths = []

fnames =  [f for f in os.listdir(IMG_FOLDER) if not (f.startswith('.') or f == 'resize')]
fixed_durations = [IMG_DISPLAY_DURATION for _ in fnames]

for fname in fnames:
    path = os.path.join(IMG_FOLDER, fname)
    try:
        img_array = resize(imread(path), output_shape=IMG_SHAPE, mode='constant')[:,:,:3]
    except ValueError:
        continue
    save_path = os.path.join(RESIZE, fname)

    final_img_paths.append(save_path)
    imsave(save_path, img_as_ubyte(img_array))


In [69]:
final_img_paths

['images/Badger//resize/8.jpg',
 'images/Badger//resize/9.jpg',
 'images/Badger//resize/14.jpg',
 'images/Badger//resize/15.jpg',
 'images/Badger//resize/17.jpg',
 'images/Badger//resize/16.jpg',
 'images/Badger//resize/12.jpg',
 'images/Badger//resize/13.jpg',
 'images/Badger//resize/11.jpg',
 'images/Badger//resize/10.jpg',
 'images/Badger//resize/21.jpg',
 'images/Badger//resize/20.jpg',
 'images/Badger//resize/22.jpg',
 'images/Badger//resize/23.jpg',
 'images/Badger//resize/18.jpg',
 'images/Badger//resize/24.jpg',
 'images/Badger//resize/25.jpg',
 'images/Badger//resize/19.jpg',
 'images/Badger//resize/4.jpg',
 'images/Badger//resize/5.jpg',
 'images/Badger//resize/7.jpg',
 'images/Badger//resize/6.jpg',
 'images/Badger//resize/2.jpg',
 'images/Badger//resize/3.jpg',
 'images/Badger//resize/1.jpg',
 'images/Badger//resize/0.jpg']

#### Creating Video
Add multiple clips end on end. Start with title clip. Add images, attach audio to images, and loop during the duration of the audio.

In [85]:
title_text = VideoClip(render_text, duration=3)

image_sequence = ImageSequenceClip(sequence=final_img_paths,
                                  durations=fixed_durations,
                                  load_images=True).\
                set_position(('center', 400)).\
                fx(vfx.loop, duration=DURATION).\
                set_audio(audio_clip).resize(VIDEO_SIZE)

video = concatenate_videoclips([title_text, image_sequence], method='compose').\
        on_color(color=BLACK, col_opacity=1)


print("Total Video Duration: ", time.strftime('%H:%M:%S', time.gmtime(video.duration)))

Total Video Duration:  00:00:09


### Save Video

In [9]:
start = datetime.now()

video.write_videofile(VID_PATH, fps=10, audio_codec="aac")

dur = datetime.now() - start
print("Video Encoding completed in time: ", dur)
os.system("say 'video writing completed'")

chunk:   6%|▋         | 116/1801 [00:00<00:01, 1159.42it/s, now=None]

Moviepy - Building video videos/Coronavirus.mp4.
MoviePy - Writing audio in CoronavirusTEMP_MPY_wvf_snd.mp4


t:   0%|          | 0/817 [00:00<?, ?it/s, now=None]                 

MoviePy - Done.
Moviepy - Writing video videos/Coronavirus.mp4



Moviepy - Done !
Moviepy - video ready videos/Coronavirus.mp4
Video Encoding completed in time:  0:02:03.656180


0

In [77]:
title_text = TextClip(title, color='white', size=VIDEO_SIZE,
         method='caption').set_duration(2)

In [78]:
title_text.size

(1920, 1080)

In [60]:
thanks = TextClip("Thanks for watching", color='white', fontsize=72, method='caption').set_duration(2)

In [71]:
thanks.size

(345, 177)

In [84]:
image_sequence.on_color()

(1920, 1080)

In [92]:
concatenate_videoclips([thanks, image_sequence]).resize(VIDEO_SIZE).duration

8.89

In [91]:
video.duration

9.89